# Data Exploration

## Setup

In [1]:
# Import libraries
import os
import pandas as pd
import requests

# Import API keys
from alyssa_config import spoonacular_key, rapidapi_key
# from lakna_config import spoonacular_key

In [2]:
# Check number of items in the Resources subdirectories, to automate file naming.
def recipe_folder():
    count = len(os.listdir('Resources/01_recipe_IDs'))-1
    return(count)

def raw_folder():
    count = len(os.listdir('Resources/02_raw_data'))-1
    return(count)

def simple_folder():
    count = len(os.listdir('Resources/03_simplified_data'))-1
    return(count)

In [3]:
# Request user input for 'initial_recipes' index of interest.
def check_index():
    try:
        index = int(input(f"Select file number (0-{recipe_folder()-1}): "))
        return index
    except ValueError as e:
        print(f"{e}: Integer required.")

## Initial Data Exploration - FOOD.COM Dataset
__Goal:__ Understand dataset format and identify attributes.

__OBSERVATIONS__
- Nutrition headings: calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat
- Will need to convert the nutrition PDV to actual values later by defining a caloric diet level.

In [4]:
# Import FOOD.COM recipes dataset
food_df = pd.read_csv('Resources/RAW_recipes.csv')
food_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
# Get food_df shape and columns
shape = food_df.shape
print(f"Shape: {shape}")
print(f"Columns: {food_df.columns}")

Shape: (231637, 12)
Columns: Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')


In [6]:
# Determine whether there are duplicate recipes by ID
dup_id = len(food_df['id'].unique())
print(f"Unique recipe IDs: {dup_id}, {shape[0] == dup_id}")

# Determine whether there are duplicate recipes by name
dup_name = len(food_df['name'].unique())
print(f"Unique recipe names: {dup_name}, {shape[0] == dup_name}")
duplicate_names = food_df.loc[food_df.duplicated(['name'])] # duplicate names should be okay
duplicate_names.head()

Unique recipe IDs: 231637, True
Unique recipe names: 230186, False


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
600,cream of mushroom soup,51922,10,59064,2003-01-21,"['15-minutes-or-less', 'time-to-make', 'course...","[472.9, 64.0, 9.0, 38.0, 28.0, 19.0, 5.0]",3,"['place walnuts , garlic water , nama shoyu , ...",if you are not a raw food vegan then try this ...,"['walnuts', 'garlic', 'water', 'shoyu', 'rosem...",8
846,10 bean soup,470575,180,1020526,2011-12-26,"['time-to-make', 'course', 'main-ingredient', ...","[228.0, 10.0, 38.0, 47.0, 42.0, 8.0, 6.0]",9,"['finely chop onion , red and green bell peppe...",we made this without the smoked turkey and ham...,"['beans', 'onion', 'red bell pepper', 'green b...",13
1314,3 bean baked beans,313237,40,407338,2008-07-11,"['60-minutes-or-less', 'time-to-make', 'course...","[329.9, 26.0, 48.0, 39.0, 33.0, 31.0, 9.0]",10,"['fry bacon until crisp , reserving 2 teaspoon...","great for cookouts, family gatherings, potluck...","['bacon', 'ground beef', 'onion', 'tomato sauc...",12
1315,3 bean salad,258846,15,607801,2007-10-13,"['15-minutes-or-less', 'time-to-make', 'course...","[502.3, 44.0, 37.0, 41.0, 24.0, 19.0, 16.0]",6,"['in medium bowl combine mustard , vinegar and...",i got this recipe from one of rachel ray's coo...,"['cut green beans', 'dijon mustard', 'sugar', ...",9
1335,3 ingredient peanut butter cookies,323810,13,138435,2008-09-07,"['15-minutes-or-less', 'time-to-make', 'course...","[197.1, 17.0, 74.0, 4.0, 11.0, 11.0, 6.0]",10,"['preheat oven to 350', 'combine the three ing...",this recipe is so simple. sometimes i put cho...,"['peanut butter', 'sugar', 'egg', 'chocolate c...",4


In [7]:
# Identify how to isolate each tag
tag_string = food_df['tags'][0].strip("[]")
tag_string = tag_string.split(', ')
tag_string[0].strip("'")

unique_tags = []
for string in food_df['tags']:
    tag_list = string.strip("[]").split(', ')
    for word_idx in range(len(tag_list)):
        tag = tag_list[word_idx].strip("'")
        if tag not in unique_tags:
            unique_tags.append(tag)
sorted(unique_tags)

['',
 '1-day-or-more',
 '15-minutes-or-less',
 '3-steps-or-less',
 '30-minutes-or-less',
 '4-hours-or-less',
 '5-ingredients-or-less',
 '60-minutes-or-less',
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.',
 'a1-sauce',
 'african',
 'american',
 'amish-mennonite',
 'angolan',
 'appetizers',
 'apples',
 'april-fools-day',
 'argentine',
 'artichoke',
 'asian',
 'asparagus',
 'australian',
 'austrian',
 'avocado',
 'bacon',
 'baja',
 'baked-beans',
 'baking',
 'bananas',
 'bar-cookies',
 'barbecue',
 'bass',
 'bean-soup',
 'beans',
 'beans-side-dishes',
 'bear',
 'beef',
 'beef-barley-soup',
 'beef-crock-pot',
 'beef-kidney',
 'beef-liver',
 'beef-organ-meats',
 'beef-ribs',
 'beef-sauces',
 'beef-sausage',
 'beginner-cook',
 'beijing',
 'belgian',
 'berries',
 'beverages',
 'birthday',
 'biscotti',
 'bisques-cream-soups',
 'black-bean-soup',
 'black-beans',
 'blueberries',
 'bok-choys',
 'brazilian',
 'bread-machine',
 'bread-pudding',
 'breads',
 'breakfast',
 '

In [8]:
# Check for meal types
meal_types = ["breakfast", "lunch", "dinner"]
for meal in meal_types:
    if meal in unique_tags:
        print(f"{meal} is a tag.")
    else:
        print(f"{meal} is not a tag.")

# Look for a similar tag to "dinner"
found = False
for tag in unique_tags:
    if (tag[:6] == "dinner"):
        print(f"{tag} is a match.")
        found = True
if (found is False):
    print("No matches found.")

# Count how many rows contain each meal type
# NOTE: the tag lists are not alphabetical, look for the first then move on? or if both in the row, drop
updated_meals = ["breakfast", "lunch", "dinner-party"]
breakfast = 0
lunch = 0
dinner = 0
multiple = 0
ping = 0

incl_recipe = []
incl_meal = []
for idx, string in food_df.iterrows():
    tag_list = string['tags'].strip("[]").split(', ')
    for word_idx in range(len(tag_list)):
        score = 0
        tag = tag_list[word_idx].strip("'")
        if (tag in updated_meals):
            ping += 1
print(ping)
#             incl_recipe.append(food_df['id'])
#             incl_meal.append(tag)

# meals_df = pd.DataFrame({'Recipe ID': incl_recipe, 'Meal Type': incl_meal})
# meals_df.head()
# print(f"Breakfast: {breakfast}")
# print(f"Lunch: {lunch}")
# print(f"Dinner: {dinner}")
# print(f"Multiple: {multiple}")
# len(food_df['tags'])

breakfast is a tag.
lunch is a tag.
dinner is not a tag.
dinner-party is a match.
75016


In [93]:
interactions_df = pd.read_csv('Resources/RAW_interactions.csv')
interactions_df

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


## Initial Data Exploration - SPOONACULAR API
__Goal:__ Test the API, check returned format and attributes.

__Process__
1. Get recipe IDs, these are needed to pass to other methods (i.e. `Recipe Information`).
2. Get recipe metadata using `Recipe Information`.
3. Save raw response to a CSV to minimise API request repetition.

### Complex Search and Recipe Information
- Complex Search: [https://spoonacular.com/food-api/docs#Search-Recipes-Complex](https://spoonacular.com/food-api/docs#Search-Recipes-Complex)
- Recipe Information: [https://spoonacular.com/food-api/docs#Get-Recipe-Information](https://spoonacular.com/food-api/docs#Get-Recipe-Information)

__OBSERVATIONS__
- `Complex Search` returns the same result every time, need to try `Random Recipes`.

In [2]:
# API REQUEST - Complex Search.
recipe_url = "https://api.spoonacular.com/recipes/complexSearch"

# Set request parameters
recipe_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of recipes.
recipe_response = requests.get(recipe_url, recipe_params).json()

# Output response to a csv
recipes_df = pd.DataFrame(recipe_response['results'])
recipes_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_0.csv', index=False)

In [26]:
# Import and display complex search results
recipes_df = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_0.csv')
recipes_df.head()

,id,title,image,imageType
0,782585,Cannellini Bean and Asparagus Salad with Mushr...,https://spoonacular.com/recipeImages/782585-31...,jpg
1,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",https://spoonacular.com/recipeImages/716426-31...,jpg
2,715497,Berry Banana Breakfast Smoothie,https://spoonacular.com/recipeImages/715497-31...,jpg
3,715415,Red Lentil Soup with Chicken and Turnips,https://spoonacular.com/recipeImages/715415-31...,jpg
4,716406,Asparagus and Pea Soup: Real Convenience Food,https://spoonacular.com/recipeImages/716406-31...,jpg


In [47]:
# Check the number of returned recipes
print(f'Number of recipes returned: {len(recipes_df)}')

Number of recipes returned: 100


In [ ]:
# API REQUEST - Recipe Information
recipe_info = []
for id in recipes_df['id']:
    info_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    info_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    info_response = requests.get(info_url, info_params).json()
    recipe_info.append(info_response)

info_df = pd.DataFrame(recipe_info)
info_df.to_csv(f'Resources/02_raw_data/info_master_0.csv')

In [ ]:
# While the API response is accessible, identify attributes of interest
recipe_info[0].keys()

# Generic metadata
recipe_info[0]['id']
recipe_info[0]['title']
recipe_info[0]['aggregateLikes'] # will need to convert this to a 5-point scale
recipe_info[0]['extendedIngredients'] # can use len() to get the number of ingredients
recipe_info[0]['weightWatcherSmartPoints'] # research the equation for this
recipe_info[0]['cuisines'] # identify recipes with non-empty results
recipe_info[0]['dishTypes'] # will need to select a simpler subset

# Nutritional values - amount, unit, percent of daily needs
recipe_info[0]['nutrition']['nutrients'][0] # Calories
recipe_info[0]['nutrition']['nutrients'][2] # Saturated Fat
recipe_info[0]['nutrition']['nutrients'][5] # Sugar
recipe_info[0]['nutrition']['nutrients'][7] # Sodium
recipe_info[0]['nutrition']['nutrients'][8] # Protein

In [85]:
# Import and display recipe information results
info_df = pd.read_csv(f'Resources/02_raw_data/info_master_0.csv')

# Get the list of column names
info_df.columns

# Display the dataframe
info_df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,dishTypes,diets,occasions,winePairing,instructions,analyzedInstructions,originalId,spoonacularSourceUrl,license,author
0,True,True,True,True,True,False,False,False,False,12,...,"['side dish', 'lunch', 'main course', 'salad',...","['gluten free', 'dairy free', 'lacto ovo veget...",[],"{'pairedWines': ['chardonnay', 'sauvignon blan...",Rinse the cannellini beans and soak for 8 hour...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/cannellini-bean-and-as...,NaN,NaN
1,True,True,True,True,True,False,True,False,False,4,...,"['side dish', 'antipasti', 'starter', 'snack',...","['gluten free', 'dairy free', 'lacto ovo veget...",[],"{'pairedWines': ['wine', 'alcoholic drink', 'w...",<ol><li><span></span>Remove the cauliflower's ...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/cauliflower-brown-rice...,CC BY-SA 3.0,NaN
2,True,False,False,False,True,False,True,False,False,15,...,"['morning meal', 'brunch', 'beverage', 'breakf...",['lacto ovo vegetarian'],[],"{'pairedWines': [], 'pairingText': '', 'produc...",<p>Take some yogurt in your favorite flavor an...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/berry-banana-breakfast...,NaN,NaN
3,False,False,True,True,True,False,True,False,False,11,...,"['lunch', 'soup', 'main course', 'main dish', ...","['gluten free', 'dairy free']","['fall', 'winter']","{'pairedWines': [], 'pairingText': '', 'produc...","To a large dutch oven or soup pot, heat the ol...","[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/red-lentil-soup-with-c...,NaN,NaN
4,True,True,True,True,True,False,False,False,False,2,...,"['antipasti', 'soup', 'starter', 'snack', 'app...","['gluten free', 'dairy free', 'paleolithic', '...","['fall', 'winter']","{'pairedWines': ['sparkling wine', 'sparkling ...",<ol><li><span></span>Chop the garlic and onion...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/asparagus-and-pea-soup...,CC BY-SA 3.0,NaN


### Random Recipes and Recipe Information
- Random Recipes: [https://spoonacular.com/food-api/docs#Get-Random-Recipes](https://spoonacular.com/food-api/docs#Get-Random-Recipes)
- Recipe Information: [https://spoonacular.com/food-api/docs#Get-Recipe-Information](https://spoonacular.com/food-api/docs#Get-Recipe-Information)

__OBSERVATIONS__
- Once converted to a dataframe, the data is not as accessible as navigating the API response, will need to create a simplified dataframe while the API response is available.
- Although this has metadata, it still does not have `Nutrition` information, will still need to run the `Recipe Information` request. Need to identify a cost-effective API request methodology.

In [38]:
random_url = "https://api.spoonacular.com/recipes/random?"

# Set request parameters
random_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of random recipe IDs.
random_response = requests.get(random_url, random_params).json()

# Output response to a csv
random_df = pd.DataFrame(random_response['recipes'])
random_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_4.csv', index=False)

In [87]:
# Import and display random recipes results
random_df = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_4.csv')
random_df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,cuisines,dishTypes,diets,occasions,instructions,analyzedInstructions,originalId,spoonacularSourceUrl,license,author
0,False,False,False,False,False,False,False,False,False,28,...,['American'],"['lunch', 'main course', 'main dish', 'dinner']",[],[],<ol><li>Preheat broiler. </li><li>Heat large f...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/tex-mex-burger-663050,NaN,NaN
1,False,False,False,False,False,False,False,False,False,39,...,[],['dessert'],[],[],<ol><li>Preheat oven to 180 degrees C</li><li>...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/chocolate-banoffee-pie...,CC BY 3.0,NaN
2,True,True,True,True,False,False,False,False,False,3,...,[],"['side dish', 'antipasti', 'salad', 'starter',...","['gluten free', 'dairy free', 'paleolithic', '...",['christmas'],Finely grate enough zest from the orange to me...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/fennel-and-orange-sala...,CC BY 3.0,NaN
3,False,False,False,True,False,False,False,False,False,12,...,['Mexican'],"['lunch', 'main course', 'main dish', 'dinner']","['dairy free', 'pescatarian']",[],In pie pan/deep dish add your Italian bread cr...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/kks-fish-tacos-648974,NaN,NaN
4,True,True,True,True,True,False,False,False,False,6,...,[],"['lunch', 'soup', 'main course', 'main dish', ...","['gluten free', 'dairy free', 'lacto ovo veget...","['fall', 'winter']",<ol><li>Rinse the beans thoroughly and place t...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",NaN,https://spoonacular.com/tuscan-white-bean-soup...,NaN,NaN


In [59]:
# Get the list of column name
random_df.columns

Index(['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy',
       'cheap', 'veryPopular', 'sustainable', 'lowFodmap',
       'weightWatcherSmartPoints', 'gaps', 'preparationMinutes',
       'cookingMinutes', 'aggregateLikes', 'healthScore', 'creditsText',
       'sourceName', 'pricePerServing', 'extendedIngredients', 'id', 'title',
       'readyInMinutes', 'servings', 'sourceUrl', 'image', 'imageType',
       'summary', 'cuisines', 'dishTypes', 'diets', 'occasions',
       'instructions', 'analyzedInstructions', 'originalId',
       'spoonacularSourceUrl', 'license', 'author'],
      dtype='object')

In [88]:
# API REQUEST - Recipe Information
recipe_info = []
for id in random_df['id']:
    info_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    info_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    info_response = requests.get(info_url, info_params).json()
    recipe_info.append(info_response)

info_df = pd.DataFrame(recipe_info)
info_df.to_csv(f'Resources/02_raw_data/info_master_4.csv')

In [89]:
# Display the dataframe
info_df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,dishTypes,diets,occasions,winePairing,instructions,analyzedInstructions,originalId,spoonacularSourceUrl,license,author
0,False,False,False,False,False,False,False,False,False,28,...,"[lunch, main course, main dish, dinner]",[],[],"{'pairedWines': ['merlot', 'malbec', 'zinfande...",<ol><li>Preheat broiler. </li><li>Heat large f...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,https://spoonacular.com/tex-mex-burger-663050,NaN,NaN
1,False,False,False,False,False,False,False,False,False,39,...,[dessert],[],[],"{'pairedWines': ['vin santo', 'late harvest ri...",<ol><li>Preheat oven to 180 degrees C</li><li>...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,https://spoonacular.com/chocolate-banoffee-pie...,CC BY 3.0,NaN
2,True,True,True,True,False,False,False,False,False,3,...,"[side dish, antipasti, salad, starter, snack, ...","[gluten free, dairy free, paleolithic, lacto o...",[christmas],"{'pairedWines': ['chardonnay', 'sauvignon blan...",Finely grate enough zest from the orange to me...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,https://spoonacular.com/fennel-and-orange-sala...,CC BY 3.0,NaN
3,False,False,False,True,False,False,False,False,False,12,...,"[lunch, main course, main dish, dinner]","[dairy free, pescatarian]",[],"{'pairedWines': ['pinot noir', 'pinot grigio',...",In pie pan/deep dish add your Italian bread cr...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,https://spoonacular.com/kks-fish-tacos-648974,NaN,NaN
4,True,True,True,True,True,False,False,False,False,6,...,"[lunch, soup, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian...","[fall, winter]","{'pairedWines': [], 'pairingText': '', 'produc...",<ol><li>Rinse the beans thoroughly and place t...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",None,https://spoonacular.com/tuscan-white-bean-soup...,NaN,NaN


### Parse API Response

__OBSERVATIONS__
- Ensure all relevant columns are accounted for before bulk requesting.
- Will need to account for different units.

In [90]:
# Metadata
recipe_id = []
recipe_name = []
recipe_likes = []
meal_type = []
cuisines = []
num_ingredients = []
ww_points = []

# Calories
calories_amt = []
calories_unit = []
calories_pct = []

# Saturated Fat
satfat_amt = []
satfat_unit = []
satfat_pct = []

# Sugar
sugar_amt = []
sugar_unit = []
sugar_pct = []

# Sodium
sodium_amt = []
sodium_unit = []
sodium_pct = []

# Protein
protein_amt = []
protein_unit = []
protein_pct = []

for idx, row in enumerate(recipe_info):
    try:
        recipe_id.append(recipe_info[idx]['id'])
        recipe_name.append(recipe_info[idx]['title'])
        recipe_likes.append(recipe_info[idx]['aggregateLikes'])
        meal_type.append(recipe_info[idx]['dishTypes'])
        cuisines.append(recipe_info[idx]['cuisines'])
        num_ingredients.append(len(recipe_info[idx]['extendedIngredients']))
        ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])
        
        calories_amt.append(recipe_info[idx]['nutrition']['nutrients'][0]['amount'])
        calories_unit.append(recipe_info[idx]['nutrition']['nutrients'][0]['unit'])
        calories_pct.append(recipe_info[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(recipe_info[idx]['nutrition']['nutrients'][2]['amount'])
        satfat_unit.append(recipe_info[idx]['nutrition']['nutrients'][2]['unit'])
        satfat_pct.append(recipe_info[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(recipe_info[idx]['nutrition']['nutrients'][5]['amount'])
        sugar_unit.append(recipe_info[idx]['nutrition']['nutrients'][5]['unit'])
        sugar_pct.append(recipe_info[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(recipe_info[idx]['nutrition']['nutrients'][7]['amount'])
        sodium_unit.append(recipe_info[idx]['nutrition']['nutrients'][7]['unit'])
        sodium_pct.append(recipe_info[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
        protein_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
        protein_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)

spoonacular_df = pd.DataFrame({
    'ID': recipe_id,
    'Name': recipe_name,
    'Likes': recipe_likes,
    'Meal Type': meal_type,
    'Cuisines': cuisines,
    'N_ingredients': num_ingredients,
    'WW Smart Points': ww_points,
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct})

spoonacular_df.to_csv(f"Resources/03_simplified_data/initial_nutrition_4.csv", index=False)

In [92]:
# Check the units columns
units_columns = ['Calories (Unit)', 'Saturated Fat (Unit)', 'Sugar (Unit)', 'Sodium (Unit)', 'Protein (Unit)']
for col in units_columns:
    check = spoonacular_df['Protein (Unit)'].unique()
    print(f"Unit check: {col} {check}")

Unit check: Calories (Unit) ['g' 'mg']
Unit check: Saturated Fat (Unit) ['g' 'mg']
Unit check: Sugar (Unit) ['g' 'mg']
Unit check: Sodium (Unit) ['g' 'mg']
Unit check: Protein (Unit) ['g' 'mg']


## Data Collection



In [23]:
# Import existing recipe IDs at requested index.
recipes_df = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_{check_index()}.csv')

# Display dataframe
recipes_df
# recipes_df['recipes'][0]

Select file number (0-3):  3


,recipes
0,"{'vegetarian': False, 'vegan': False, 'glutenF..."
1,"{'vegetarian': False, 'vegan': False, 'glutenF..."
2,"{'vegetarian': True, 'vegan': False, 'glutenFr..."
3,"{'vegetarian': True, 'vegan': False, 'glutenFr..."
4,"{'vegetarian': False, 'vegan': False, 'glutenF..."
...,...
95,"{'vegetarian': True, 'vegan': False, 'glutenFr..."
96,"{'vegetarian': True, 'vegan': False, 'glutenFr..."
97,"{'vegetarian': True, 'vegan': False, 'glutenFr..."
98,"{'vegetarian': True, 'vegan': False, 'glutenFr..."


In [ ]:
# Data Collection - Spoonacular API, using Random Recipe
random_url = "https://api.spoonacular.com/recipes/random?"

# Set request parameters
random_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of random recipe IDs.
random_response = requests.get(random_url, random_params).json()

# Output response to a csv
random_df = pd.DataFrame(random_response['recipes'])
random_df.to_csv(f'Resources/Recipe_IDs/random_results_{recipe_folder}.csv', index=False)

In [14]:
# Import recipe raw data of interest
# random_df = pd.read_csv(f'Resources/
check_index()
random_df = pd.read_csv(f'Resources/Recipe_IDs/

Select file number:  3


3

In [11]:
random_list = []
for index in range(len(random_response['recipes'])):
    random_list.append(random_response['recipes'][index]['id'])

In [12]:
random_list

[645384,
 636271,
 639137,
 634229,
 660283,
 633595,
 663879,
 634986,
 716431,
 659143,
 636300,
 641110,
 638586,
 655609,
 1003210,
 644965,
 632071,
 638035,
 665172,
 716351,
 642585,
 658129,
 637335,
 636415,
 1098355,
 634389,
 716370,
 647799,
 659929,
 801377,
 660231,
 665642,
 653670,
 649062,
 657719,
 646217,
 640355,
 716425,
 633091,
 638940,
 641644,
 638603,
 635182,
 653785,
 663164,
 658418,
 657823,
 652825,
 665185,
 652696,
 632790,
 637999,
 638693,
 1056442,
 648983,
 650225,
 632101,
 648275,
 631747,
 715380,
 633386,
 715544,
 635756,
 648408,
 651225,
 632151,
 716429,
 716422,
 639728,
 660609,
 635126,
 647577,
 650546,
 635795,
 654959,
 661500,
 658967,
 654430,
 651823,
 639637,
 636973,
 634295,
 663054,
 635418,
 637210,
 663987,
 665744,
 647210,
 640720,
 631783,
 67565,
 640730,
 637488,
 716361,
 647016,
 644846,
 715497,
 639333,
 641727,
 632590]

In [13]:
for id in recipes_df['id']:
    print(id)

782585
716426
715497
715415
716406
644387
715446
782601
795751
766453
716627
664147
640941
715495
794349
716381
660306
756814
715769
642129
642605
636589
646738
715540
663559
662670
640062
715421
648320
716432
798400
782600
652417
639535
729366
661925
794538
649931
636787
641975
652423
715447
716311
715543
157344
658509
716195
716361
765011
660228
716408
661340
782622
715424
659135
639851
664547
633921
658579
715385
641057
655575
632269
637162
982371
1095753
660405
715523
632252
639891
646651
716330
640318
642085
716437
664090
636602
716276
649944
646043
634437
655235
652393
640828
769774
715533
639411
636228
661259
664680
715391
636230
632812
633942
632347
975070
649886
659927
716217
636608


In [13]:
# Get recipe and nutrition information
nutrition_list = []
# for id in recipes_df['id']:
for id in random_list:
    nutrition_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    nutrition_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    nutrition_response = requests.get(nutrition_url, nutrition_params).json()
    nutrition_list.append(nutrition_response)

In [21]:
nutrition_list

[{'vegetarian': True,
  'vegan': False,
  'glutenFree': True,
  'dairyFree': False,
  'veryHealthy': False,
  'cheap': False,
  'veryPopular': False,
  'sustainable': False,
  'lowFodmap': False,
  'weightWatcherSmartPoints': 3,
  'gaps': 'no',
  'preparationMinutes': 10,
  'cookingMinutes': 30,
  'aggregateLikes': 2,
  'healthScore': 6,
  'creditsText': 'Jen West',
  'sourceName': 'Pink When',
  'pricePerServing': 61.66,
  'extendedIngredients': [{'id': 11507,
    'aisle': 'Produce',
    'image': 'sweet-potato.png',
    'consistency': 'SOLID',
    'name': 'sweet potatoes',
    'nameClean': 'sweet potato',
    'original': '3 medium sweet potatoes',
    'originalName': 'sweet potatoes',
    'amount': 3.0,
    'unit': 'medium',
    'meta': [],
    'measures': {'us': {'amount': 3.0,
      'unitShort': 'medium',
      'unitLong': 'mediums'},
     'metric': {'amount': 3.0, 'unitShort': 'medium', 'unitLong': 'mediums'}}},
   {'id': 1001,
    'aisle': 'Milk, Eggs, Other Dairy',
    'image': '

In [14]:
nutrition_master = pd.DataFrame(nutrition_list)
nutrition_master.to_csv("Resources/nutrition_master_3.csv", index=False)

In [15]:
recipe_id = []
recipe_name = []
recipe_likes = []
meal_type = []
cuisines = []
num_ingredients = []
ww_points = []

calories_amt = []
calories_unit = []
calories_pct = []

satfat_amt = []
satfat_unit = []
satfat_pct = []

sugar_amt = []
sugar_unit = []
sugar_pct = []

sodium_amt = []
sodium_unit = []
sodium_pct = []

protein_amt = []
protein_unit = []
protein_pct = []


for idx, row in enumerate(nutrition_list):
    try:
        recipe_id.append(nutrition_list[idx]['id'])
        recipe_name.append(nutrition_list[idx]['title'])
        recipe_likes.append(nutrition_list[idx]['aggregateLikes'])
        meal_type.append(nutrition_list[idx]['dishTypes'])
        cuisines.append(nutrition_list[idx]['cuisines'])
        num_ingredients.append(len(nutrition_list[idx]['extendedIngredients']))
        ww_points.append(nutrition_list[idx]['weightWatcherSmartPoints'])
        
        # Second to last '0' index is for CALORIES
        calories_amt.append(nutrition_list[idx]['nutrition']['nutrients'][0]['amount'])
        calories_unit.append(nutrition_list[idx]['nutrition']['nutrients'][0]['unit'])
        calories_pct.append(nutrition_list[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(nutrition_list[idx]['nutrition']['nutrients'][2]['amount'])
        satfat_unit.append(nutrition_list[idx]['nutrition']['nutrients'][2]['unit'])
        satfat_pct.append(nutrition_list[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(nutrition_list[idx]['nutrition']['nutrients'][5]['amount'])
        sugar_unit.append(nutrition_list[idx]['nutrition']['nutrients'][5]['unit'])
        sugar_pct.append(nutrition_list[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(nutrition_list[idx]['nutrition']['nutrients'][7]['amount'])
        sodium_unit.append(nutrition_list[idx]['nutrition']['nutrients'][7]['unit'])
        sodium_pct.append(nutrition_list[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(nutrition_list[idx]['nutrition']['nutrients'][8]['amount'])
        protein_unit.append(nutrition_list[idx]['nutrition']['nutrients'][8]['unit'])
        protein_pct.append(nutrition_list[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)

spoonacular_df = pd.DataFrame({
    'ID': recipe_id,
    'Name': recipe_name,
    'Likes': recipe_likes,
    'Meal Type': meal_type,
    'Cuisines': cuisines,
    'N_ingredients': num_ingredients,
    'WW Smart Points': ww_points,
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct})

unit_check = spoonacular_df['Protein (Unit)'].unique()
unit_check

spoonacular_df.to_csv("Resources/initial_nutrition_6.csv", index=False)

33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
